In [164]:
import pandas as pd
import numpy as np
import random as rm
import re

In [165]:
sharks = pd.read_csv('../Data/attacks.csv',encoding='utf-8',encoding_errors= 'ignore')

In [166]:
sharks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [167]:
#Eliminamos los duplicados que tras cierta exploracion coinciden con la mayoria de valores nulos
sharks = sharks.drop_duplicates() 

In [168]:
pd.set_option('display.max_columns',None)
sharks.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [169]:
sharks['Case Number'].value_counts() #podemos ver que como mucho, salvando los 0s que no dejan de ser nulos, solo hay duplicados por pares

0               8
1920.00.00.b    2
1966.12.26      2
2014.08.02      2
1990.05.10      2
               ..
1999.09.05      1
1999.09.10      1
1999.09.16      1
1999.09.18      1
xx              1
Name: Case Number, Length: 6287, dtype: int64

In [170]:
sharks['Case Number'].fillna('0',inplace=True)

In [171]:
sharks['Case Number'].value_counts()

0               10
2009.12.18       2
1920.00.00.b     2
2013.10.05       2
1983.06.15       2
                ..
1999.09.05       1
1999.09.10       1
1999.09.16       1
1999.09.18       1
xx               1
Name: Case Number, Length: 6287, dtype: int64

In [172]:
def Fix_Points_ID(x):

  '''
  Función que quita los puntos del ID para dejar un número realmente identificatorio del suceso
  Los duplicados serán corregidos en otra función
  '''
  x = str(x)     #Me quito el problema de que algunos sean float(presuntamente los 0s que tambien solucionaremos mas adelante)
  if '.' in x:
         
         p = x.split('.')
         x = ''
         for n in range(len(p)):
            x +=p[n]
         x +='0'  #Sera el default para los duplicados que cambiaré en otra función
  return x
def Fix_Letters_ID(x):

  '''
  Función que quita las letras del ID para dejar un número realmente identificatorio del suceso
  Los duplicados serán corregidos en otra función
  '''

  if 'a' in x or 'b' in x:
         if 'a' in x:
             p = x.split('a')
             x = ''
             for n in range(len(p)):
                 x +=p[n]
         elif 'b' in x:
             p = x.split('b')
             x = ''
             for n in range(len(p)):
                 x +=p[n]    
         
  return x

sharks['Case Number'] = sharks['Case Number'].apply(Fix_Points_ID)
sharks['Case Number'] = sharks['Case Number'].apply(Fix_Letters_ID)
sharks.head(10)     
        

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,201806250,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,201806180,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,201806090,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,201806080,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,201806040,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
5,201806030,03-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,NaN,"No injury, board bitten",N,NaN,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b-FlatRock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.b,2018.06.03.b,6298.0,NaN,NaN
6,201806030,03-Jun-2018,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.a,2018.06.03.a,6297.0,NaN,NaN
7,201805270,27-May-2018,2018.0,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,52,Minor injury to foot. PROVOKED INCIDENT,N,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.27,2018.05.27,6296.0,NaN,NaN
8,201805260,26-May-2018,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.b,2018.05.26.b,6295.0,NaN,NaN
9,201805260,26-May-2018,2018.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",Standing,male,M,12,Minor injury to foot,N,14h00,NaN,"K. McMurray, Tracking Sharks.com",2018.05.26.a-DaytonaBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.a,2018.05.26.a,6294.0,NaN,NaN


In [173]:
no_dupl = []
p = ''
for i in range(len(sharks['Case Number'])): #Eliminamos duplicados que no sean 0s
    if sharks['Case Number'].iloc[i] != '0':
        if sharks['Case Number'].iloc[i] in no_dupl and sharks['Case Number'].iloc[i] != '0':
            p = sharks['Case Number'].iloc[i][:-1]
            p +='1'
            if p in no_dupl:
                p = sharks['Case Number'].iloc[i][:-1]
                p +='2'
                no_dupl.append(p)
            else:
                no_dupl.append(p)
        else:
            no_dupl.append(sharks['Case Number'].iloc[i])
    else:
        no_dupl.append(sharks['Case Number'].iloc[i])
sharks['Case Number'] = no_dupl
pepito = []
sharks['Case Number'].value_counts()


0            10
201806250     1
195705000     1
195702020     1
195702050     1
             ..
199909180     1
199909240     1
199909290     1
199910010     1
xx            1
Name: Case Number, Length: 6303, dtype: int64

In [174]:
#Igualo la longitud de todos para que tengan el mismo formato y modifico los duplicados para que vuelvan a ser unicos
def Igualar_longitud(x):
    if len(x) != 9:
        if len(x) > 9:
            while len(x)>9:
                x = x[:-1]
        elif len(x)< 9:
            while len(x)<9:
                x+='0'
    if x in pepito:
        b = 0
        while x in pepito and b != 9:
            if re.match(r"\d",x[0]): 
                l = int(x[0])
                l +=1
                if l == 10:
                    l = 0
                l = str(l)
                x = l+x[1:]
            elif re.match(r"\d",x[-1]):
                l = int(x[-1])
                l+=1
                if l == 10:
                    l = 0
                l = str(l)
                x = x[:-1] + l
            b+=1
            
                
    pepito.append(x)
    return x
sharks['Case Number'] = sharks['Case Number'].apply(Igualar_longitud)
sharks['Case Number'].value_counts()


900000000    2
201806250    1
195704130    1
19570000i    1
19570000j    1
            ..
199909050    1
199909100    1
199909160    1
199909180    1
xx0000000    1
Name: Case Number, Length: 6311, dtype: int64

In [175]:
sharks.loc[sharks.loc[sharks['Case Number']=='900000000'].index[0], 'Case Number'] = '900000001'
sharks['Case Number'].value_counts()

201806250    1
19570000g    1
19570000i    1
19570000j    1
19570000k    1
            ..
199909050    1
199909100    1
199909160    1
199909180    1
xx0000000    1
Name: Case Number, Length: 6312, dtype: int64

In [176]:
#Elimino la fila con valor xx que está vacía y no aporta
sharks.drop(sharks.loc[sharks['Case Number']=='xx0000000'].index,inplace=True)

In [177]:
#Renombro la columna por Attack ID que me parece mejor y más explicativa
nombres_col=[]
nombres_col = list(sharks.columns)
nombres_col[0] = 'Attack ID'
sharks.columns = nombres_col
sharks.head()

,Attack ID,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,201806250,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,201806180,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,201806090,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,201806080,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,201806040,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [178]:
sharks['Date'].value_counts()

1957                          11
1942                           9
1956                           8
1958                           7
1950                           7
                              ..
01-Dec-1994                    1
Reported      10-Dec-1994      1
11-Dec-1994                    1
13-Dec-1994                    1
1845-1853                      1
Name: Date, Length: 5433, dtype: int64

In [179]:
sharks.loc[sharks['Date']=='1957']

,Attack ID,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
4211,19570000k,1957,1957.0,Invalid,SRI LANKA,Western Province,Colombo Harbor,Fishing,F. L. Fernando,M,NaN,"FATAL, but shark involvement not confirmed",NaN,NaN,Shark involvement prior to death was not confi...,R. I. DeSilva,1957.00.00.k-Fernando.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.k,1957.00.00.k,2092.0,NaN,NaN
4212,19570000j,1957,1957.0,Unprovoked,USA,Florida,"Fort Pierce, St. Lucie County",Bathing,R. Nauth,NaN,NaN,Injured by shark,N,NaN,1.5 m [5'] shark,R.F. Hutton,1957.00.00.j-NV-Nauth.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.j,1957.00.00.j,2091.0,NaN,NaN
4213,19570000i,1957,1957.0,Invalid,CUBA,Havana Province,Cojimar,NaN,an infant,NaN,2 to 3 months,"FATAL, but was it an accident or infanticide?",NaN,NaN,3.7 m [12'] shark,"F. Poli, p.23-24",1957.00.00.i-baby,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.i,1957.00.00.i,2090.0,NaN,NaN
4214,19570000h,1957,1957.0,Boat,CUBA,Havana Province,Cojimar,Fishing,"boat, occupant: Portuondo",NaN,NaN,"No injury to occupant, shark stuck boat",N,NaN,3.7 m [12'] shark,"F. Poli, pp.21-24",1957.00.00.h-boat-Portuondo.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.h,1957.00.00.h,2089.0,NaN,NaN
4215,19570000g,1957,1957.0,Unprovoked,CUBA,Havana Province,Havana,Fishing for sharks,William Bolster,M,18,"FATAL, became entangled in fishing line and pu...",Y,NaN,NaN,"F. Poli, p.9",1957.00.00.g-Bolster.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.g,1957.00.00.g,2088.0,NaN,NaN
4216,19570000f,1957,1957.0,Unprovoked,USA,Florida,"Miami, Miami-Dade County",Helmet diving in Miami Seaquarium,Jim Kline,M,NaN,"Shark struck helmet, no injury",N,NaN,Bull shark,"R. Carras, H.D. Baldridge, p.181",1957.00.00.f-Kline.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.f,1957.00.00.f,2087.0,NaN,NaN
4217,19570000e,1957,1957.0,Unprovoked,FIJI,Kadavu,Great Astrolabe Reef,Fishing,Semesa Vasu,M,30,Right foot lacerated,N,NaN,NaN,S.B. Brown,1957.00.00.e-Vasu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.e,1957.00.00.e,2086.0,NaN,NaN
4218,19570000d,1957,1957.0,Unprovoked,IRAN,Karun River,"near Band Misan in Shustar, 420 km from the sea",NaN,Mr. Paniry,M,NaN,Survived,N,NaN,Bull shark,B. Coad,1957.00.00.d-Paniry.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.d,1957.00.00.d,2085.0,NaN,NaN
4219,195700000,1957,1957.0,Unprovoked,PAPUA NEW GUINEA,"Abau Subdistrict,Central Province",Mailu area,Fishing,"male, from Laluoro",M,NaN,"FATAL, multiple injuries",Y,NaN,NaN,"A. Bleakley; A.M. Rapson, p.149",1957.00.00.b-Mailu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.b,1957.00.00.b,2084.0,NaN,NaN
4220,19570000c,1957,1957.0,Unprovoked,IRAN,Karun River,Hesamabad,NaN,Mr. Falah,M,NaN,Survived,N,NaN,Bull shark,B. Coad & F. Papahn,1957.00.00.c-Falah.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1957.00.00.c,1957.00.00.c,2083.0,NaN,NaN


In [180]:
#sharks['Date'].fillna('0',inplace=True)

In [181]:
def Contar_nulos(sharks):
 nan_cols = sharks.isna().sum()

 display(nan_cols[nan_cols>0])
 

In [182]:
sharks['Date'].fillna('xxxx',inplace=True) #relleno los pocos nulos que hay de esta forma para mantener el formato luego
sharks['Date'] = sharks['Date'].astype(str) #cambio todos los valores a string para trabajar con ellos

In [183]:
def Fix_s(x):

  '''
  Función que quita la s a los años que quieren representar una decada
  '''
  p = ''   
  if 's' in x:
         
         p = x[:-1]
         return p
  return x
def Fix_reported(x):

  '''
  Función que quita reported
  '''
  p = ''   
  if 'Reported' in x:
         
         p = x.split()
         try:
              
              return p[1]
         except:
              return 'No date'
              
         
  return x
sharks['Date'] = sharks['Date'].apply(Fix_reported)
sharks['Date'] = sharks['Date'].apply(Fix_s)


In [184]:

sharks = sharks[~sharks['Date'].str.contains("No date")]
sharks = sharks[~sharks['Date'].str.contains("Before")]

sharks['Date'].value_counts()


#Plantee si borrar aquellos datos que fueran solo el año pero quedan demasiados pocos datos

1957           11
1942            9
xxxx            9
1950            9
1960            8
               ..
13-Dec-1994     1
18-Dec-1994     1
30-Dec-1994     1
02-Jan-1995     1
19-Sep-1976     1
Name: Date, Length: 5320, dtype: int64

In [185]:

def Fix_format_date(x):

  '''
  Función que rellenará de xx los valores de mes y dia desconocidos de modo que todas las fechas tengan el mismo
  formato
  '''
  if ' - ' in x:
       x = 'xx-xx-'+x.split(' - ')[0]
  elif '-' in x:
       try:  
          int(x[0])
          if len(x.split('-')[0]) == 4:
             x = 'xx-xx-'+x.split('-')[0]
       except:
          x = 'xx-'+x
  else:
     x = 'xx-xx-'+x 
  return x
sharks['Date'] = sharks['Date'].apply(Fix_format_date)
sharks['Date'].value_counts()

xx-xx-1941     11
xx-xx-1957     11
xx-xx-1950     10
xx-xx-xxxx      9
xx-xx-1960      9
               ..
24-Jan-1995     1
xx-Feb-1995     1
19-Feb-1995     1
05-Mar-1995     1
26-Nov-1976     1
Name: Date, Length: 5305, dtype: int64

In [186]:
#De la columna anterior voy a aprovechar las columnas que tengo vacías al final del dataframe
#para separar año (columna que ya existe), mes y día
nombres_col[22] = 'Month'
nombres_col[23] = 'Day'
sharks.columns = nombres_col
sharks.head()

,Attack ID,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Month,Day
0,201806250,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,201806180,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,201806090,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,201806080,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,201806040,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [187]:


for i in range(len(sharks['Month'])):

  '''
  Bucle que rellena la columna Month
  '''
  sp = sharks['Date'].iloc[i].split('-')
  sharks['Month'].iloc[i] = sp[1]

for i in range(len(sharks['Day'])):

  '''
  Bucle que rellena la columna Day
  '''
  sp = sharks['Date'].iloc[i].split('-')
  sharks['Day'].iloc[i] = sp[0]

sharks.head(10)
  
              
         
 

C:\Users\pacot\AppData\Local\Temp\ipykernel_25944\905297956.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Month'].iloc[i] = sp[1]
C:\Users\pacot\AppData\Local\Temp\ipykernel_25944\905297956.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Day'].iloc[i] = sp[0]


,Attack ID,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Month,Day
0,201806250,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,Jun,25
1,201806180,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,Jun,18
2,201806090,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,Jun,09
3,201806080,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,Jun,08
4,201806040,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,Jun,04
5,201806030,03-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,NaN,"No injury, board bitten",N,NaN,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b-FlatRock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.b,2018.06.03.b,6298.0,Jun,03
6,201806031,03-Jun-2018,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.a,2018.06.03.a,6297.0,Jun,03
7,201805270,27-May-2018,2018.0,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,52,Minor injury to foot. PROVOKED INCIDENT,N,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.27,2018.05.27,6296.0,May,27
8,201805260,26-May-2018,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.b,2018.05.26.b,6295.0,May,26
9,201805261,26-May-2018,2018.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",Standing,male,M,12,Minor injury to foot,N,14h00,NaN,"K. McMurray, Tracking Sharks.com",2018.05.26.a-DaytonaBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.a,2018.05.26.a,6294.0,May,26


In [188]:
#Pasamos el año a entero
sharks['Year'].fillna(0,inplace=True)
sharks['Year'] = sharks['Year'].astype(int)
sharks['Year'] = sharks['Year'].replace(0,'xxxx') 


In [189]:
#Reordeno las columnas en función a todos los cambios que iré haciendo en el DF
sharks = sharks.reindex(columns= ['Attack ID', 'Date', 'Year', 'Month', 'Day', 'Time', 'Country', 'Area', 'Location', 'Type','Case Number.1',	'Case Number.2', 'Activity', 'Name', 'Sex ','Age', 'Injury', 'Fatal (Y/N)',	'original order','Species ','Investigator or Source', 'pdf', 'href formula', 'href'])
nombres_col = ['Attack ID', 'Date', 'Year', 'Month', 'Day', 'Time', 'Country', 'Area', 'Location', 'Provoked/Unprovoked','Boat Related',	'Sea Disaster Related', 'Activity', 'Name', 'Sex','Age', 'Injury', 'Fatal (Y/N)','Shark could be not involved','Specie',  'Investigator or Source', 'pdf', 'href formula', 'href']
sharks.head()

,Attack ID,Date,Year,Month,Day,Time,Country,Area,Location,Type,Case Number.1,Case Number.2,Activity,Name,Sex,Age,Injury,Fatal (Y/N),original order,Species,Investigator or Source,pdf,href formula,href
0,201806250,25-Jun-2018,2018,Jun,25,18h00,USA,California,"Oceanside, San Diego County",Boating,2018.06.25,2018.06.25,Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,6303.0,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
1,201806180,18-Jun-2018,2018,Jun,18,14h00 -15h00,USA,Georgia,"St. Simon Island, Glynn County",Unprovoked,2018.06.18,2018.06.18,Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,6302.0,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
2,201806090,09-Jun-2018,2018,Jun,09,07h45,USA,Hawaii,"Habush, Oahu",Invalid,2018.06.09,2018.06.09,Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,6301.0,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
3,201806080,08-Jun-2018,2018,Jun,08,NaN,AUSTRALIA,New South Wales,Arrawarra Headland,Unprovoked,2018.06.08,2018.06.08,Surfing,male,M,NaN,Minor injury to lower leg,N,6300.0,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
4,201806040,04-Jun-2018,2018,Jun,04,NaN,MEXICO,Colima,La Ticla,Provoked,2018.06.04,2018.06.04,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,6299.0,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...


In [190]:
sharks['Fatal (Y/N)'] = sharks['Fatal (Y/N)'].replace('UNKNOWN',np.nan)
sharks['Fatal (Y/N)'] = sharks['Fatal (Y/N)'].replace('2017',np.nan)
#Tranformo a nulos los valores que tienen "valor" pero que realmente no lo tienen

In [191]:
#Arreglo los valores nulos de la fatalidad de los ataques en base a la estadistica encontrada
#Al investigar en internet 


valores_reemplazo = ['Y', 'N']

pesos = [0.2,0.8]

valores_aleatorios = rm.choices(valores_reemplazo, pesos, k=sharks['Fatal (Y/N)'].isnull().sum())

sharks.loc[sharks['Fatal (Y/N)'].isnull(), 'Fatal (Y/N)'] = valores_aleatorios

In [192]:
sharks['Fatal (Y/N)'] = sharks['Fatal (Y/N)'].replace('y','Y')
sharks['Fatal (Y/N)'] = sharks['Fatal (Y/N)'].replace(' N','N')
sharks['Fatal (Y/N)'] = sharks['Fatal (Y/N)'].replace('M','N')
sharks['Fatal (Y/N)'] = sharks['Fatal (Y/N)'].replace('N ','N')
#Les doy el valor correcto a los valores que no lo tenían
sharks['Fatal (Y/N)'].value_counts()  

N    4744
Y    1473
Name: Fatal (Y/N), dtype: int64

In [193]:
Contar_nulos(sharks)

Time                      3272
Country                     57
Area                       443
Location                   523
Type                        13
Case Number.1                9
Case Number.2                9
Activity                   539
Name                       215
Sex                        568
Age                       2752
Injury                      36
original order               2
Species                   2779
Investigator or Source      26
pdf                          9
href formula                10
href                         9
dtype: int64

In [194]:
sharks['Time'].value_counts()

Afternoon      187
11h00          128
Morning        120
12h00          109
15h00          108
              ... 
00h30            1
14h37            1
10h07            1
13h53            1
19h00-20h00      1
Name: Time, Length: 365, dtype: int64

In [195]:
sharks['Time'] = sharks['Time'].replace(['Dusk','dusk','Dawn','Early morning','Early Morning','Morning','Mid-morning','Late morning','Noon','Midday','After noon','Afternoon','Late afternoon','Late Afternoon','Evening','Night','Midnight','Late night','2 hrs before sunset'],['03h00','03h00','05h00','04h00','04h00','08h00','10h00','11h00','12h00','12h00','15h00','15h00','17h00','17h00','20h00','22h00','00h00','02h00','22h00'])
sharks['Time'] = sharks['Time'].replace('--',np.nan)
sharks['Time'] = sharks['Time'].replace('11h01 -time of ship sinking',np.nan)
sharks['Time'] = sharks['Time'].replace('Ship aban-doned at 03h10',np.nan)
#elimino todos los tiempos aproximados por palabra, igualo "--" a nulo que viene a ser lo mismo y corrijo dos casos especificos

In [196]:
sharks['Time'].fillna('xxhxx',inplace=True)

In [197]:
filtrado = sharks.loc[sharks['Time'].str.contains('-')] #Miro los casos de horas por rangos
print (filtrado)

      Attack ID         Date  Year Month Day            Time  \
1     201806180  18-Jun-2018  2018   Jun  18   14h00  -15h00   
356   201509240  24-Sep-2015  2015   Sep  24     10h45-11h15   
536   201407051   5-Jul-2014  2014   Jul   5   07h00 - 08h00   
537   201407030   3-Jul-2014  2014   Jul   3     18h15-18h30   
704   201302090  09-Feb-2013  2013   Feb  09   09h00 - 09h30   
1193  200811060  06-Nov-2008  2008   Nov  06  10h00 -- 11h00   
1881  200202160  16-Feb-2002  2002   Feb  16    09h00 -10h00   
3801  196208120  12-Aug-1962  1962   Aug  12   14h00 - 15h00   
3823  196205120  12-May-1962  1962   May  12   03h45 - 04h00   
3998  196004140  14-Apr-1960  1960   Apr  14   09h00 - 09h30   
5963  186212220  22-Dec-1862  1862   Dec  22  06h00 -- 07h00   
6091  183004300  30-Apr-1830  1830   Apr  30     17h00-18h00   
6105  18260520R  20-May-1826  1826   May  20     19h00-20h00   

               Country             Area  \
1                  USA          Georgia   
356              

In [198]:
sharks['Time'] = sharks['Time'].replace('14h00  -15h00','14h30')
def Fix_ranges(x):

  '''
  Función que eliminará los rangos de las horas 
  '''
  p = ''
  if '--'in x:
       p = x.split('--')
       l = p[0].split('h')
       x = l[0]+'h'+'30'
       return x
          
  if '-' in x:
         
         p = x.split('-')
         print(p)
         if p[0][-2] == '0':
               if '3' in p[1]:
                    l = p[0].split('h')
                    x = l[0]+'h'+'15'
               elif p[1][-2] == '0':
                    l = p[0].split('h')
                    x = l[0]+'h'+'30'
         elif p[0][-2] == '4':
              if p[1][-2] == '1':
                   x = '11h00'
              else:
                   x = '03h52'
         elif p[0][-2] == '1':
              x = '18h22'
   # Convertir strings completamente en palabras a 'Unknown'
  if not re.match(r'^\d+|\d+.\d+$', x):
        
        return 'xxhxx'

    # Reemplazar comillas, signos de interrogación por nada
  x = re.sub(r'["?]', '', x)

    # Reemplazar letras por nada
  if (x[2] != 'h'):
     x = re.sub(r'[a-zA-Z]', '', x)
     if len(x) == 4:
          f = list(x)
          x = f[0]+f[1]+'h'+f[2]+f[3]
  if len(x) == 6:
       x = x[:-1]
  if x[2] == ':':
     f = x.split(':')
     x = f[0]+'h'+f[1]
  j = x.split()
  if 'Sunset' in x:
       u = list(x)
       x = u[0]+u[1]+u[2]+u[3]+u[4]
  try:
     if j[1] == '/' or j[1] == '&' or j[1] == 'to' or j[1] == 'or':
          x = j[0]
  except:
       return x      
  return x

sharks['Time'] = sharks['Time'].apply(Fix_ranges)
#Casos especiales

sharks['Time'] = sharks['Time'].replace('03h45 - 04h00','03h52')


['10h45', '11h15']
['07h00 ', ' 08h00']
['18h15', '18h30']
['09h00 ', ' 09h30']
['09h00 ', '10h00']
['14h00 ', ' 15h00']
['03h45 ', ' 04h00']
['09h00 ', ' 09h30']
['17h00', '18h00']
['19h00', '20h00']


In [199]:
#Renombro la columna a hora para que quede más claro
nombres_col[5] = 'Hour'
sharks.columns = nombres_col
sharks.head()

,Attack ID,Date,Year,Month,Day,Hour,Country,Area,Location,Provoked/Unprovoked,Boat Related,Sea Disaster Related,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Shark could be not involved,Specie,Investigator or Source,pdf,href formula,href
0,201806250,25-Jun-2018,2018,Jun,25,18h00,USA,California,"Oceanside, San Diego County",Boating,2018.06.25,2018.06.25,Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,6303.0,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
1,201806180,18-Jun-2018,2018,Jun,18,14h30,USA,Georgia,"St. Simon Island, Glynn County",Unprovoked,2018.06.18,2018.06.18,Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,6302.0,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
2,201806090,09-Jun-2018,2018,Jun,09,07h45,USA,Hawaii,"Habush, Oahu",Invalid,2018.06.09,2018.06.09,Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,6301.0,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
3,201806080,08-Jun-2018,2018,Jun,08,xxhxx,AUSTRALIA,New South Wales,Arrawarra Headland,Unprovoked,2018.06.08,2018.06.08,Surfing,male,M,NaN,Minor injury to lower leg,N,6300.0,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
4,201806040,04-Jun-2018,2018,Jun,04,xxhxx,MEXICO,Colima,La Ticla,Provoked,2018.06.04,2018.06.04,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,6299.0,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...


In [200]:

sharks['Country'].value_counts()

USA                       2215
AUSTRALIA                 1323
SOUTH AFRICA               571
PAPUA NEW GUINEA           132
NEW ZEALAND                127
                          ... 
NORTH SEA                    1
RED SEA / INDIAN OCEAN       1
BRITISH WEST INDIES          1
ADMIRALTY ISLANDS            1
CEYLON (SRI LANKA)           1
Name: Country, Length: 207, dtype: int64

In [201]:
for i in range(len(sharks['Country'])):

  '''
  Bucle que rellena los nulos del pais/región si el area tiene dato
  '''
  if sharks['Country'].iloc[i] is np.nan:
     sharks['Country'].iloc[i] = sharks['Area'].iloc[i]

#Algunos datos especificos
sharks['Country'].iloc[4639] = 'Hong Kong'
sharks['Area'].iloc[4639] = 'Hong Kong'
sharks['Area'].iloc[3605] = 'Florida Strait'
sharks['Country'].iloc[6155] = 'Barbados'
sharks['Area'].iloc[6155] = 'Carlisle Bay'
sharks['Country'].iloc[5810] = 'Argentina'
sharks['Area'].iloc[5810] = 'Santa Cruz'


#sharks.loc[sharks['Country'].isna()]


C:\Users\pacot\AppData\Local\Temp\ipykernel_25944\2619594468.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Country'].iloc[i] = sharks['Area'].iloc[i]
C:\Users\pacot\AppData\Local\Temp\ipykernel_25944\2619594468.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Country'].iloc[4639] = 'Hong Kong'
C:\Users\pacot\AppData\Local\Temp\ipykernel_25944\2619594468.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shar

In [202]:
 #Tras una investigación determino que USA es el lugar donde es más común que sucedan los ataques
 #Además coincide con la moda con lo cual relleno los nulos con este dato

sharks['Country'].fillna('USA',inplace=True)
sharks['Country'] = sharks['Country'].str.title()
sharks['Country'].value_counts()

Usa                   2255
Australia             1323
South Africa           571
Papua New Guinea       132
New Zealand            127
                      ... 
Grand Cayman             1
Mexico                   1
Sudan?                   1
The Balkans              1
Ceylon (Sri Lanka)       1
Name: Country, Length: 214, dtype: int64

In [203]:
filtrado = sharks.loc[sharks['Country'].str.contains('/')] #Miro en cuantos duda entre paises
print (filtrado)

      Attack ID         Date  Year Month Day   Hour  \
2292  199607231  23-Jul-1996  1996   Jul  23  18h00   
3957  19601000d  xx-Oct-1960  1960   Oct  xx  xxhxx   
4140  195809130  13-Sep-1958  1958   Sep  13  14h00   
4619  194508190  19-Aug-1945  1945   Aug  19  xxhxx   
4696  194211130  13-Nov-1942  1942   Nov  13  xxhxx   
4934  19340826R  26-Aug-1934  1934   Aug  26  xxhxx   
5978  18610212R  12-Feb-1861  1861   Feb  12  xxhxx   

                           Country                               Area  \
2292                Egypt / Israel         South Sinai, Gulf of Aqaba   
3957        Red Sea / Indian Ocean  Enroute from Suez to Aden (Yemen)   
4140    Andaman / Nicobar Islandas                                NaN   
4619                   Iran / Iraq                Shatt-al-Arab River   
4696     Solomon Islands / Vanuatu                                NaN   
4934               Italy / Croatia                                NaN   
5978  Equatorial Guinea / Cameroon              

In [204]:
#Me quedaré con el primer pais/región por ser ocasionalmente más restrictivo

def Fix_region(x):

  '''
  Función que eliminará las dudas entre paises divididos por /
  '''
  p = ''  
  if '/' in x:
         
         p = x.split('/')
         x = p[0]
              
         
  return x

sharks['Country'] = sharks['Country'].apply(Fix_region)
sharks['Country'] = sharks['Country'].replace('Usa','USA')
sharks['Country'] = sharks['Country'].replace('Sudan?','Sudan')

filtrado = sharks.loc[sharks['Country'].str.contains('/')] #Miro en cuantos duda entre paises
print (filtrado)

Empty DataFrame
Columns: [Attack ID, Date, Year, Month, Day, Hour, Country, Area, Location, Provoked/Unprovoked, Boat Related, Sea Disaster Related, Activity, Name, Sex, Age, Injury, Fatal (Y/N), Shark could be not involved, Specie, Investigator or Source, pdf, href formula, href]
Index: []


In [205]:
def Fix_parentesis(x):
    p = ''
    if '(' in x:
        p = x.split('(')
        x = p[1][:-1]
    
    
    return x
sharks['Country'] = sharks['Country'].apply(Fix_parentesis)

In [206]:
#Finalmente renombro la columna a Country/Region ya que no siempre es un pais el dato que aparece
#Sin ir más lejos puede ser en mar abierto
nombres_col[6] = 'Country/Region'
sharks.columns = nombres_col
sharks.head()

,Attack ID,Date,Year,Month,Day,Hour,Country/Region,Area,Location,Provoked/Unprovoked,Boat Related,Sea Disaster Related,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Shark could be not involved,Specie,Investigator or Source,pdf,href formula,href
0,201806250,25-Jun-2018,2018,Jun,25,18h00,USA,California,"Oceanside, San Diego County",Boating,2018.06.25,2018.06.25,Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,6303.0,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
1,201806180,18-Jun-2018,2018,Jun,18,14h30,USA,Georgia,"St. Simon Island, Glynn County",Unprovoked,2018.06.18,2018.06.18,Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,6302.0,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
2,201806090,09-Jun-2018,2018,Jun,09,07h45,USA,Hawaii,"Habush, Oahu",Invalid,2018.06.09,2018.06.09,Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,6301.0,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
3,201806080,08-Jun-2018,2018,Jun,08,xxhxx,Australia,New South Wales,Arrawarra Headland,Unprovoked,2018.06.08,2018.06.08,Surfing,male,M,NaN,Minor injury to lower leg,N,6300.0,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
4,201806040,04-Jun-2018,2018,Jun,04,xxhxx,Mexico,Colima,La Ticla,Provoked,2018.06.04,2018.06.04,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,6299.0,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...


In [207]:
#Rellenamos los nulos 
for i in range(len(sharks['Area'])):
  '''
  Bucle que rellena los nulos del Area si el Location tiene dato
  Prefiero tener el dato de la localización como area (que en muchos casos es un dato representativo)
  Antes de poner "desconocido"
  '''
  if sharks['Area'].iloc[i] is np.nan:
     sharks['Area'].iloc[i] = sharks['Location'].iloc[i]

sharks['Area'].fillna('Unknown',inplace=True)
sharks['Location'].fillna('Unknown',inplace=True)

C:\Users\pacot\AppData\Local\Temp\ipykernel_25944\562506109.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Area'].iloc[i] = sharks['Location'].iloc[i]


In [208]:
Contar_nulos(sharks)

Provoked/Unprovoked              13
Boat Related                      9
Sea Disaster Related              9
Activity                        539
Name                            215
Sex                             568
Age                            2752
Injury                           36
Shark could be not involved       2
Specie                         2779
Investigator or Source           26
pdf                               9
href formula                     10
href                              9
dtype: int64

In [209]:
sharks.loc[sharks['Location']=='Carlisle Bay']

,Attack ID,Date,Year,Month,Day,Hour,Country/Region,Area,Location,Provoked/Unprovoked,Boat Related,Sea Disaster Related,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Shark could be not involved,Specie,Investigator or Source,pdf,href formula,href
6155,174212170,17-Dec-1742,1742,Dec,17,xxhxx,USA,Carlisle Bay,Carlisle Bay,Unprovoked,1742.12.17,1742.12.17,Swimming,2 impressed seamen,M,NaN,FATAL,Y,148.0,NaN,"C. Moore, GSAF",1742.12.17-AdviceSeamen.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
6160,170303260,26-Mar-1703,1703,Mar,26,22h00,Barbados,Southwest coast,Carlisle Bay,Unprovoked,1703.03.26,1703.03.26,Swimming,"Samuel Jennings, a deserter from the British f...",M,19,"Hand and foot severely bitten, surgically ampu...",N,143.0,NaN,"W.R.Cutter, Vol.1, p.252",1703.03.26-Jennings.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...


In [210]:
#Dividí (aprovechando código de más arriba) esta columna en 3 dejando provocado/no provocado, relacionado con embarcacion
#y relacionado con un desastre marino
pd.unique(sharks['Provoked/Unprovoked'])


array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan, 'Boat', 'Boatomg'], dtype=object)

In [211]:
sharks['Provoked/Unprovoked'] = sharks['Provoked/Unprovoked'].replace( ['Boatomg','Boating'],'Boat')
sharks['Provoked/Unprovoked'] = sharks['Provoked/Unprovoked'].replace( ['Questionable','Invalid',np.nan],'Unprovoked')

In [212]:
pd.unique(sharks['Provoked/Unprovoked'])

array(['Boat', 'Unprovoked', 'Provoked', 'Sea Disaster'], dtype=object)

In [213]:
#Relleno las nuevas columnas de barco y desastre marino involucrados a la par que relleno la de provocado o no 
#con no provocado pues porcentualmente la gran mayoría de ataques no lo son
for i in range(len(sharks['Provoked/Unprovoked'])):
    if sharks['Provoked/Unprovoked'].iloc[i] == 'Sea Disaster':
        sharks['Sea Disaster Related'].iloc[i] = 'Y'
        sharks['Boat Related'].iloc[i] = 'N'
        sharks['Provoked/Unprovoked'].iloc[i] = 'Unprovoked'
    elif sharks['Provoked/Unprovoked'].iloc[i] == 'Boat':
        sharks['Boat Related'].iloc[i] = 'Y'
        sharks['Sea Disaster Related'].iloc[i] = 'N'
        sharks['Provoked/Unprovoked'].iloc[i] = 'Unprovoked'
    elif sharks['Provoked/Unprovoked'].iloc[i] == 'Provoked' or sharks['Provoked/Unprovoked'].iloc[i] == 'Unprovoked':
        sharks['Sea Disaster Related'].iloc[i] = 'N'
        sharks['Boat Related'].iloc[i] = 'N'
sharks.head()        

C:\Users\pacot\AppData\Local\Temp\ipykernel_25944\3381381134.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Boat Related'].iloc[i] = 'Y'
C:\Users\pacot\AppData\Local\Temp\ipykernel_25944\3381381134.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Sea Disaster Related'].iloc[i] = 'N'
C:\Users\pacot\AppData\Local\Temp\ipykernel_25944\3381381134.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Provoked

,Attack ID,Date,Year,Month,Day,Hour,Country/Region,Area,Location,Provoked/Unprovoked,Boat Related,Sea Disaster Related,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Shark could be not involved,Specie,Investigator or Source,pdf,href formula,href
0,201806250,25-Jun-2018,2018,Jun,25,18h00,USA,California,"Oceanside, San Diego County",Unprovoked,Y,N,Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,6303.0,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
1,201806180,18-Jun-2018,2018,Jun,18,14h30,USA,Georgia,"St. Simon Island, Glynn County",Unprovoked,N,N,Standing,AdysonMcNeely,F,11,Minor injury to left thigh,N,6302.0,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
2,201806090,09-Jun-2018,2018,Jun,09,07h45,USA,Hawaii,"Habush, Oahu",Unprovoked,N,N,Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,6301.0,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
3,201806080,08-Jun-2018,2018,Jun,08,xxhxx,Australia,New South Wales,Arrawarra Headland,Unprovoked,N,N,Surfing,male,M,NaN,Minor injury to lower leg,N,6300.0,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
4,201806040,04-Jun-2018,2018,Jun,04,xxhxx,Mexico,Colima,La Ticla,Provoked,N,N,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,6299.0,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...


In [214]:
sharks['Activity'].value_counts()

Surfing                                                 970
Swimming                                                860
Fishing                                                 422
Spearfishing                                            333
Bathing                                                 159
                                                       ... 
Fishing, trying to catch the end of his fishing line      1
Washing clothes on stairs                                 1
Washing cooking pans                                      1
Spearfishing / freediving                                 1
Wooden fishing boat                                       1
Name: Activity, Length: 1506, dtype: int64

In [215]:
#Relleno los nulos con los dos valores mas comunes (la moda)
valores_reemplazo = ['Fishing', 'Surfing']

pesos = [0.5,0.5]

valores_aleatorios = rm.choices(valores_reemplazo, pesos, k=sharks['Activity'].isnull().sum())

sharks.loc[sharks['Activity'].isnull(), 'Activity'] = valores_aleatorios

In [216]:
#Si hay un bote en actividad lo introduzco en el dato de barco
for i in range(len(sharks['Activity'])):
    if 'boat' in sharks['Activity'].iloc[i] or 'Boat' in sharks['Activity'].iloc[i] or 'dinghy' in sharks['Activity'].iloc[i] or 'Dinghy' in sharks['Activity'].iloc[i]:
        sharks['Boat Related'].iloc[i] = 'Y'

C:\Users\pacot\AppData\Local\Temp\ipykernel_25944\3456156495.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Boat Related'].iloc[i] = 'Y'


In [217]:

def Simplifica(x):
    '''
    Esta función limpia la columna de actividades y si ciertas actividades aparecen en la descripción las
    unifica para que sea más claro
    '''
    x = str(x)
    if 'Fishing' in x or 'fishing' in x:
        x = 'Fishing'
    elif 'Bathing' in x or 'bathing' in x:
        x = 'Bathing'
    elif 'Surfing' in x or 'surfing' in x:
        x = 'Surfing'
    elif 'Swimming' in x or 'swimming' in x:
        x = 'Swimming'
    elif 'diving' in x or 'Diving' in x:
        x = 'Free Diving'
    elif 'snorkeling' in x or 'Snorkeling' in x:
        x = 'Snorkeling'
    elif 'paddling' in x or 'Paddling' in x:
        x = 'Paddling'
    elif 'Sinking' in x or 'sinking' in x or 'sank' in x or 'Sank' in x:
        x = 'Sinking'
    elif 'boat' in x or 'Boat'in x:
        x = 'Sailing' 
    return x
sharks['Activity'] = sharks['Activity'].apply(Simplifica)
sharks['Activity'].value_counts()

Surfing                                                                           1415
Fishing                                                                           1402
Swimming                                                                          1081
Free Diving                                                                        506
Bathing                                                                            188
                                                                                  ... 
Hauling dead shark aboard, when another shark leapt out of the water & bit him       1
Argentine Air Force C-54                                                             1
Arsinoe, a French tanker                                                             1
Attempting to anesthetize shark                                                      1
Spent 8 days in dinghy                                                               1
Name: Activity, Length: 690, dtype: int64

In [218]:
def quitar_comillas(x):
    x = re.sub(r'["?.]','', x)
    return x
sharks['Activity'] = sharks['Activity'].apply(quitar_comillas)
pd.unique(sharks['Activity'])

array(['Paddling', 'Standing', 'Surfing', 'Free Diving', 'Swimming',
       'Fishing', 'Walking', 'Feeding sharks', 'Boogie boarding',
       'Paddle-skiing', 'Body boarding', 'Stand-Up Paddleboarding',
       'Wading', 'Cleaning fish', 'Snorkeling', 'Sailing', 'Bathing',
       'Kayaking', 'Canoeing', 'SUP', 'Skimboarding', 'Touching a shark',
       'Attempting to lasso a shark', 'Photo shoot', 'Paddle boarding',
       'Kakaying', 'Washing hands', 'Grabbing shark for a selfie',
       'Tagging sharks', 'Surf skiing ', 'Floating', 'Surfng',
       'SUP Foil boarding', 'Lobstering', 'Floating in tube',
       'Teasing a shark', 'Surf-skiing', 'Feeding stingrays',
       'Sea disaster', 'Photographing fish', 'Treading water',
       'Body boarding ', 'Playing in the water', 'Body Boarding',
       'Playing', 'Attempting to rescue a shark',
       'Photographing the shark', 'Hand feeding sharks',
       'Sitting in the water', 'Transatlantic Rowing', 'Rowing',
       'Filming', 'Standin

In [219]:
sharks['Name'].value_counts()

male                    530
female                   95
boy                      21
2 males                  16
boat                     14
                       ... 
Craig Ruth                1
Gilbert Bui Van Minh      1
Barry Pasonski            1
Heather Van Olst          1
Jules Patterson           1
Name: Name, Length: 5172, dtype: int64

In [220]:
#Si hay un bote en nombre lo introduzco en el dato de barco
sharks['Name'].fillna('Unknown',inplace=True)
for i in range(len(sharks['Name'])):
    if 'boat' in sharks['Name'].iloc[i] or 'Boat' in sharks['Name'].iloc[i] or 'dinghy' in sharks['Name'].iloc[i] or 'Dinghy' in sharks['Name'].iloc[i]:
        sharks['Boat Related'].iloc[i] = 'Y'

C:\Users\pacot\AppData\Local\Temp\ipykernel_25944\780429211.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Boat Related'].iloc[i] = 'Y'


In [221]:
#Reemplazo aquellos datos que no son un nombre por unknown
def male_female(x):
    if 'male' in x or 'Male' in x or 'boy' in x or 'sailor' in x or 'Sailor' in x or 'diver' in x or 'Diver'in x or 'a ' in x or '2' in x:
        return 'Unknown'
    return x
sharks['Name'] = sharks['Name'].apply(male_female)
sharks['Name'] = sharks['Name'].replace(['dinghy','unknown','fisherman','Unidentified','Anonymous','anonymous','child','Child','girl','Boy','boat','Boat'],'Unknown')
sharks['Name'].value_counts()

Unknown            1716
M.C.                  3
Andre Hartman         3
John Williams         3
Rodney Orr            2
                   ... 
Mark Taylor           1
Peck Euwer            1
Jevan Wright          1
Cameron Bayes         1
Jules Patterson       1
Name: Name, Length: 4455, dtype: int64

In [222]:
sharks['Sex'].value_counts()

M      5016
F       627
M         2
N         2
lli       1
.         1
Name: Sex, dtype: int64

In [223]:
#Limpio los valores claros y luego relleno nulos
sharks['Sex'] = sharks['Sex'].replace(['M ','N'],'M')
sharks['Sex'] = sharks['Sex'].replace(['lli','.'],'Unknown')
sharks['Sex'].fillna('Unknown',inplace=True)

In [224]:
Contar_nulos(sharks)

Age                            2752
Injury                           36
Shark could be not involved       2
Specie                         2779
Investigator or Source           26
pdf                               9
href formula                     10
href                              9
dtype: int64

In [225]:
pd.unique(sharks['Age'])

array(['57', '11', '48', nan, '18', '52', '15', '12', '32', '10', '21',
       '34', '30', '60', '33', '29', '54', '41', '37', '56', '19', '25',
       '69', '38', '55', '35', '46', '45', '14', '40s', '28', '20', '24',
       '26', '49', '22', '7', '31', '17', '40', '13', '42', '3', '8',
       '50', '16', '82', '73', '20s', '68', '51', '39', '58', 'Teen',
       '47', '61', '65', '36', '66', '43', '60s', '9', '72', '59', '6',
       '27', '64', '23', '71', '44', '62', '63', '70', '18 months', '53',
       '30s', '50s', 'teen', '77', '74', '28 & 26', '5', '86', '18 or 20',
       '12 or 13', '46 & 34', '28, 23 & 30', 'Teens', '36 & 26',
       '8 or 10', '84', ' ', '30 or 36', '21 & ?', '75', '33 or 37',
       'mid-30s', '23 & 20', ' 30', '7      &    31', ' 28', '20?',
       "60's", '32 & 30', '16 to 18', '87', '67', 'Elderly', 'mid-20s',
       'Ca. 33', '74 ', '45 ', '21 or 26', '20 ', '>50', '18 to 22',
       'adult', '9 & 12', '? & 19', '9 months', '25 to 35', '23 & 26',
      

In [226]:
#CASOS PARTICULARES
sharks['Age'] = sharks['Age'].replace(['F','X','MAKE LINE GREEN','  ','A.M.'],np.nan)
sharks['Age'].fillna('Unknown',inplace=True)
sharks['Age'] = sharks['Age'].replace(['teen','Teen'],'14')
sharks['Age'] = sharks['Age'].replace(['young','"young"'],'20')
sharks['Age'] = sharks['Age'].replace('Elderly','80')
sharks['Age'] = sharks['Age'].replace('Ca. 33','33')
sharks['Age'] = sharks['Age'].replace('>50','51')
sharks['Age'] = sharks['Age'].replace('7      &    31','7 & 31')
sharks['Age'] = sharks['Age'].replace('?    &   14','? & 14')
############################################
def quitar_rangos(x):
    if 'adult' in x or 'middle' in x:
        x = '40'
        return x
    if ('to' in x or 'or' in x) and 'month' not in x:
        j = x.split()
        x = j[0]
    if x[-1] == 's':
        x = x[-3] + x[-2]
    if x[-1] == ' ':
        x = x[:-1]
    return x
sharks['Age'] = sharks['Age'].apply(quitar_rangos)
#Casos particulares que por alguna razón no se resolvían arriba
sharks['Age'] = sharks['Age'].replace(['th','en',''],np.nan)
sharks['Age'].fillna('Unknown',inplace=True)

In [227]:
pd.unique(sharks['Age'])

array(['57', '11', '48', 'Unknown', '18', '52', '15', '12', '32', '10',
       '21', '34', '30', '60', '33', '29', '54', '41', '37', '56', '19',
       '25', '69', '38', '55', '35', '46', '45', '14', '40', '28', '20',
       '24', '26', '49', '22', '7', '31', '17', '13', '42', '3', '8',
       '50', '16', '82', '73', '68', '51', '39', '58', '47', '61', '65',
       '36', '66', '43', '9', '72', '59', '6', '27', '64', '23', '71',
       '44', '62', '63', '70', '53', '77', '74', '28 & 26', '5', '86',
       '46 & 34', '28, 23 & 30', '36 & 26', '84', '21 & ?', '75',
       '23 & 20', ' 30', '7 & 31', ' 28', '20?', "0'", '32 & 30', '87',
       '67', '80', '9 & 12', '? & 19', '23 & 26', '1', '33 & 37',
       '37, 67, 35, 27,  ? & 27', '21, 34,24 & 35', '30 & 32', '50 & 30',
       '17 & 35', '34 & 19', '33 & 26', ' 43', '81', '78', '17 & 16',
       'Both 11', '36 & 23', '? & 14', '2'], dtype=object)

In [228]:
#Las columnas de investigador/fuente y sus pdfs además de ser poco relevantes (si a caso el primer enlace al dato puede ser interesante)
#también son dificiles de limpiar ya que no tenemos forma no solo de comprobar la veracidad del dato sino tampoco
#su formato. Por esto decido solo rellenar los nulos
sharks['href formula'].fillna('Unknown',inplace=True)
sharks['Investigator or Source'].fillna('Unknown',inplace=True)
sharks['pdf'].fillna('Unknown',inplace=True)

In [229]:
sharks['href'] = sharks['href formula']

In [230]:
sharks['Injury'].value_counts()

FATAL                                                                   777
Survived                                                                 94
Foot bitten                                                              83
No injury                                                                81
Leg bitten                                                               71
                                                                       ... 
Leg & hands bitten                                                        1
Grabbed small shark & it bit him  PROVOKED INCIDENT                       1
FATAL, arm bitten                                                         1
Multiple injuries                                                         1
FATAL. "Shark bit him in half, carrying away the lower extremities"       1
Name: Injury, Length: 3700, dtype: int64

In [231]:
#Me doy cuenta de algunos valores que aun puedo rellenar de años y lo hago
for i in range(len(sharks['Year'])):

  '''
  Bucle que rellena la columna Year
  '''
  if (sharks['Year'].iloc[i] == 'xxxx'):
    try:
      sp = sharks['Date'].iloc[i].split('-')
      sharks['Year'].iloc[i] = sp[2]
    except:
      sharks['Year'].iloc[i] = 'xxxx'

C:\Users\pacot\AppData\Local\Temp\ipykernel_25944\2632682777.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Year'].iloc[i] = sp[2]
C:\Users\pacot\AppData\Local\Temp\ipykernel_25944\2632682777.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Year'].iloc[i] = 'xxxx'


In [232]:
#Saco la posible info que hay, relevante para otras columnas y transformo la columna a binario (Herida/No herida)
sharks['Injury'].fillna('Unknown',inplace=True)
for i in range(len(sharks['Injury'])):
    if 'FATAL' in sharks['Injury'].iloc[i] or 'Fatal' in sharks['Injury'].iloc[i] or 'fatal' in sharks['Injury'].iloc[i]:
        sharks['Fatal (Y/N)'].iloc[i] = 'Y'
    if 'Survived' in sharks['Injury'].iloc[i] or 'survived' in sharks['Injury'].iloc[i] or 'SURVIVED' in sharks['Injury'].iloc[i]:
        sharks['Fatal (Y/N)'].iloc[i] = 'N'
    if 'Provoked' in sharks['Injury'].iloc[i] or 'provoked' in sharks['Injury'].iloc[i] or 'PROVOKED' in sharks['Injury'].iloc[i]:
        sharks['Provoked/Unprovoked'].iloc[i] ='Provoked'
    if 'Unprovoked' in sharks['Injury'].iloc[i] or 'unprovoked' in sharks['Injury'].iloc[i] or 'UNPROVOKED' in sharks['Injury'].iloc[i]:
        sharks['Provoked/Unprovoked'].iloc[i] ='Unprovoked'
    if 'Boat' in sharks['Injury'].iloc[i] or 'boat' in sharks['Injury'].iloc[i] or 'BOAT' in sharks['Injury'].iloc[i]: 
        sharks['Boat Related'].iloc[i] ='Y'
    if 'No injury' in sharks['Injury'].iloc[i] or 'no injury' in sharks['Injury'].iloc[i] or 'NO INJURY' in sharks['Injury'].iloc[i]:
        sharks['Injury'].iloc[i] = 'No injury'
    else:
        sharks['Injury'].iloc[i] = 'Injury'

C:\Users\pacot\AppData\Local\Temp\ipykernel_25944\1136757032.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Injury'].iloc[i] = 'No injury'
C:\Users\pacot\AppData\Local\Temp\ipykernel_25944\1136757032.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Injury'].iloc[i] = 'Injury'
C:\Users\pacot\AppData\Local\Temp\ipykernel_25944\1136757032.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Provoked/Unprov

In [233]:
sharks['Injury'].value_counts()

Injury       5443
No injury     774
Name: Injury, dtype: int64

In [234]:
sharks['Specie'].value_counts()

White shark                                           163
Shark involvement prior to death was not confirmed    105
Invalid                                               101
Shark involvement not confirmed                        88
Tiger shark                                            71
                                                     ... 
Blacktip shark, 5' to 6'                                1
3' shark, possibly  a blacktip or spinner shark         1
Possibly  a blacktip or spinner shark                   1
12' sandtiger shark                                     1
12' tiger shark                                         1
Name: Specie, Length: 1534, dtype: int64

In [235]:
#Convierto los nulos en unknown y elimino las comillas y las ? 

sharks['Specie'].fillna('Unknown',inplace=True)
def quitar_comillas(x):
    x = re.sub(r'["?.]','', x)
    return x
sharks['Specie'] = sharks['Specie'].apply(quitar_comillas)

In [236]:
#Guardo todos los posibles datos que me pueda dar la columna y despues almaceno
#si es posible de forma simple el tipo de tiburon

for i in range(len(sharks['Specie'])):
    sharks['Shark could be not involved'].iloc[i] = 'N'
    if 'FATAL' in sharks['Specie'].iloc[i] or 'Fatal' in sharks['Specie'].iloc[i] or 'fatal' in sharks['Specie'].iloc[i]:
        sharks['Fatal (Y/N)'].iloc[i] = 'Y'
    if 'Survived' in sharks['Specie'].iloc[i] or 'survived' in sharks['Specie'].iloc[i] or 'SURVIVED' in sharks['Specie'].iloc[i]:
        sharks['Fatal (Y/N)'].iloc[i] = 'N'
    if 'Provoked' in sharks['Specie'].iloc[i] or 'provoked' in sharks['Specie'].iloc[i] or 'PROVOKED' in sharks['Specie'].iloc[i]:
        sharks['Provoked/Unprovoked'].iloc[i] ='Provoked'
    if 'Unprovoked' in sharks['Specie'].iloc[i] or 'unprovoked' in sharks['Specie'].iloc[i] or 'UNPROVOKED' in sharks['Specie'].iloc[i]:
        sharks['Provoked/Unprovoked'].iloc[i] ='Unprovoked'
    if 'Boat' in sharks['Specie'].iloc[i] or 'boat' in sharks['Specie'].iloc[i] or 'BOAT' in sharks['Specie'].iloc[i]: 
        sharks['Boat Related'].iloc[i] ='Y'
    if 'not confirmed' in sharks['Specie'].iloc[i] or 'questionable' in sharks['Specie'].iloc[i] or 'Questionable' in sharks['Specie'].iloc[i]:
        sharks['Shark could be not involved'].iloc[i] = 'Y'
        sharks['Specie'].iloc[i] = 'Unknown'
    elif 'shark' in sharks['Specie'].iloc[i] or 'shark,' in sharks['Specie'].iloc[i] or 'sharks' in sharks['Specie'].iloc[i] or 'shark,' in sharks['Specie'].iloc[i]:
        
        o = sharks['Specie'].iloc[i].split()
        for d in range(len(o)):
            if o[d] == 'shark':
                sharks['Specie'].iloc[i] = o[d-1] + ' ' + o[d]
            elif o[d] == 'shark,' or o[d] == 'sharks' or o[d] == 'sharks,':
                sharks['Specie'].iloc[i] = o[d-1] + ' ' + o[d][:-1]
    else:
        sharks['Specie'].iloc[i] = 'Unknown'


C:\Users\pacot\AppData\Local\Temp\ipykernel_25944\3551861217.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['Shark could be not involved'].iloc[i] = 'N'


In [237]:
#Hago unos pequeños cambios uniendo especies que son la misma y cambiando casos particulares
#También hago la suposición de que m shark es mako shark siendo el tiburon mas común con m en inglés
sharks['Specie'] = sharks['Specie'].replace('white shark','White shark')
sharks['Specie'] = sharks['Specie'].replace('m shark','Mako shark')
sharks['Specie'] = sharks['Specie'].replace('Hammerhead shark+O2356','Hammerhead shark')
sharks['Specie'] = sharks['Specie'].replace('Hammerhead shark+O2356','Hammerhead shark')
sharks['Specie'] = sharks['Specie'].replace('sandshark','Sand shark')


sharks['Specie'] = sharks['Specie'].str.title()

sharks['Specie'] = sharks['Specie'].replace('Small Catsharks','Cat shark')
sharks['Specie'] = sharks['Specie'].replace('A Shark','Unknown')
sharks['Specie'] = sharks['Specie'].replace('Whaler shark','Copper shark')

sharks['Specie'].value_counts()


Unknown               3342
White Shark            620
Tiger Shark            257
Mako Shark             182
Bull Shark             171
                      ... 
Silvertip Shark          1
Gaffed Shark             1
150-Lb Shark             1
Two Shark                1
Black-Tipped Shark       1
Name: Specie, Length: 236, dtype: int64

In [238]:
#Aquellos valores que solo me dan un numero, ya sea de lo largo o pesado que sea el tiburon lo paso a unknown porque no se que especie es
def si_numero_unknown(x):
    x = re.sub(r'\d+','Unknown', x)
    t = ['Unknown','Unidentified','Large','Other','Same','Little','Cm','Of','No Shark','Young','Juvenile','Kg','Female','Two','From','Captured','The','Another','Small']
    for i in range(len(t)):
        if t[i] in x:
            x = 'Unknown'
    return x
sharks['Specie'] = sharks['Specie'].apply(si_numero_unknown)
sharks['Specie'].value_counts()

Unknown               4208
White Shark            620
Tiger Shark            257
Mako Shark             182
Bull Shark             171
                      ... 
Soupfin Shark            1
Gaffed Shark             1
Silvertip Shark          1
Believed Shark           1
Black-Tipped Shark       1
Name: Specie, Length: 78, dtype: int64

In [239]:
pd.unique(sharks['Specie'])

array(['White Shark', 'Unknown', 'Mako Shark', 'Tiger Shark',
       'Lemon Shark', 'Bull Shark', 'Reef Shark', 'Nurse Shark',
       'Wobbegong Shark', 'Blacktip Shark', 'Galapagos Shark',
       'Blue Shark', 'Cookiecutter Shark', 'Spinner Shark',
       'Whitetip Shark', 'Sandtiger Shark', 'Seven-Gill Shark',
       'Sevengill Shark', 'Angel Shark', 'Dogfish Shark', 'Whaler Shark',
       'Silky Shark', 'Hammerhead Shark', 'Raggedtooth Shark',
       'Goblin Shark', 'Metre Shark', "Oceanic Whitetip Shark'",
       'Sandbar Shark', 'Cow Shark', 'Salmon Shark', 'Porbeagle Shark',
       'Jackson Shark', 'Zambesi Shark', 'Thresher Shark', 'Whale Shark',
       'Cutter Shark', 'Dusky Shark', 'Smoothhound Shark',
       'Basking Shark', 'Sand Shark', 'Cat shark', 'Copper Shark',
       'Tipped Shark', 'Brown Shark', 'Grey-Colored Shark',
       'Captive Shark', 'Colored Shark', 'Bonnethed Shark',
       'Finned Shark', 'Blackfin Shark', 'Soupfin Shark', 'Leopard Shark',
       'Grey Shar

In [240]:
sharks['Shark could be not involved'].value_counts()

N    5911
Y     306
Name: Shark could be not involved, dtype: int64

In [242]:
def pasar_a_bool(x):
    if x == 'Y' or x == 'Provoked' or x == 'Injury':
        return True
    elif x == 'N' or x == 'Unprovoked' or x == 'No injury':
        return False
    else:
        return 'Unknown'
    
sharks['Boat Related'] = sharks['Boat Related'].apply(pasar_a_bool)
sharks['Provoked/Unprovoked'] = sharks['Provoked/Unprovoked'].apply(pasar_a_bool)
sharks['Sea Disaster Related'] = sharks['Sea Disaster Related'].apply(pasar_a_bool)
sharks['Fatal (Y/N)'] = sharks['Fatal (Y/N)'].apply(pasar_a_bool)
sharks['Shark could be not involved'] = sharks['Shark could be not involved'].apply(pasar_a_bool)
sharks['Injury'] = sharks['Injury'].apply(pasar_a_bool)

In [243]:
#Un overview de como ha quedado la tabla finalmente
sharks.head()

,Attack ID,Date,Year,Month,Day,Hour,Country/Region,Area,Location,Provoked/Unprovoked,Boat Related,Sea Disaster Related,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Shark could be not involved,Specie,Investigator or Source,pdf,href formula,href
0,201806250,25-Jun-2018,2018,Jun,25,18h00,USA,California,"Oceanside, San Diego County",False,True,False,Paddling,Julie Wolfe,F,57,False,False,False,White Shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
1,201806180,18-Jun-2018,2018,Jun,18,14h30,USA,Georgia,"St. Simon Island, Glynn County",False,False,False,Standing,AdysonMcNeely,F,11,True,False,False,Unknown,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
2,201806090,09-Jun-2018,2018,Jun,09,07h45,USA,Hawaii,"Habush, Oahu",False,False,False,Surfing,John Denges,M,48,True,False,False,Unknown,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
3,201806080,08-Jun-2018,2018,Jun,08,xxhxx,Australia,New South Wales,Arrawarra Headland,False,False,False,Surfing,Unknown,M,Unknown,True,False,False,Mako Shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
4,201806040,04-Jun-2018,2018,Jun,04,xxhxx,Mexico,Colima,La Ticla,True,False,False,Free Diving,Gustavo Ramos,M,Unknown,True,False,False,Tiger Shark,A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...


In [253]:
pd.unique(sharks['Year'])

array([2018, 2017, '2017', 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009,
       2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998,
       1997, 1996, 1995, 1984, 1994, 1993, 1992, 1991, 1990, 1989, 1969,
       1988, 1987, 1986, 1985, 1983, 1982, 1981, 1980, 1979, 1978, 1977,
       1976, 1975, 1974, 1973, 1972, 1971, 1970, 1968, 1967, 1966, 1965,
       1964, 1963, 1962, 1961, 1960, 1959, 1958, 1957, 1956, 1955, 1954,
       1953, 1952, 1951, 1950, 1949, 1948, 1848, 1947, 1946, 1945, 1944,
       1943, 1942, 1941, 1940, 1939, 1938, 1937, 1936, 1935, 1934, 1933,
       1932, 1931, 1930, 1929, 1928, 1927, 1926, 1925, 1924, 1923, 1922,
       1921, 1920, 1919, 1918, 1917, 1916, 1915, 1914, 1913, 1912, 1911,
       1910, 1909, 1908, 1907, 1906, 1905, 1904, 1903, 1902, 1901, 1900,
       1899, 1898, 1897, 1896, 1895, 1894, 1893, 1892, 1891, 1890, 1889,
       1888, 1887, 1886, 1885, 1884, 1883, 1882, 1881, 1880, 1879, 1878,
       1877, 1876, 1875, 1874, 1873, 1872, 1871, 

In [252]:
def year_to_int(x):
    return int(x)
sharks['Year'] = sharks['Year'].apply(year_to_int)

ValueError: invalid literal for int() with base 10: 'Ca. 214 B.C.'

In [244]:
sharks.to_csv('../Data/SharkAttacks.csv', index=False)